In [1]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
beer_data = cPickle.load(open('../../beer_data/beer_vec_ds_df10.p', 'rb'))

Using TensorFlow backend.


[]


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

idxs = val_idxs
H = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = map(lambda s : list(idxs).index(s), a0)
    a1 = map(lambda s : list(idxs).index(s), a1)
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
train_X = beer_data.X[train_idxs]
train_X = map(lambda s : [beer_data.idx2word[t] for t in s if t > 0], train_X)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False)
train_TF = tfidf.fit_transform(train_X)

In [5]:
val_X = beer_data.X[val_idxs]
val_X = map(lambda s : [beer_data.idx2word[t] for t in s if t > 0], val_X)

In [6]:
val_TF = tfidf.transform(val_X)

In [13]:
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
#embedds_n = normalize(np.array(val_TF.todense()), 'l2')
#scores = np.dot(embedds_n, embedds_n.T)
scores = cosine_similarity(embedds_n)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [39]:
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.588893125367
0 0.597270081065
3 0.583869723277
2 0.587069684288


In [14]:
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.622527668018
0 0.630764199813
3 0.615854123934
2 0.619540610685


In [80]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=200)

In [81]:
train_lsa = lsa.fit_transform(train_C)

In [82]:
val_lsa = lsa.transform(val_C)

In [83]:
from sklearn.preprocessing import normalize
embedds_n = normalize(val_lsa, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000
scores = np.array(scores)

In [44]:
#800
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.588232224084
0 0.596819010281
3 0.583551483894
2 0.586798876777


In [84]:
#200 C
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.556527375244
0 0.562210688798
3 0.55356365904
2 0.555587495722


In [15]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(tokenizer=lambda i:i, lowercase=False)
train_C = count.fit_transform(train_X)
val_C = count.transform(val_X)

In [17]:
train_lda = lda.fit_transform(train_C)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [18]:
val_lda = lda.transform(val_C)

In [19]:
from sklearn.preprocessing import normalize
embedds_n = normalize(val_lda, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000
scores = np.array(scores)

In [63]:
#10 LDA
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.697612091001
0 0.696654492985
3 0.69404681481
2 0.695302691346


In [20]:
#10 LDA 14k
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.688605689378
0 0.691113059652
3 0.686241107155
2 0.687701227426
